In [1]:
import pandas as pd

rasff=pd.read_csv(r"F:\Final_project\rasff_new2.csv")

In [2]:
rasff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27397 entries, 0 to 27396
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   reference              27397 non-null  float64
 1   category               27397 non-null  object 
 2   type                   27397 non-null  object 
 3   subject                27397 non-null  object 
 4   date                   27397 non-null  object 
 5   notifying_country      27397 non-null  object 
 6   classification         27397 non-null  object 
 7   risk_decision          27397 non-null  object 
 8   distribution           18759 non-null  object 
 9   forAttention           14966 non-null  object 
 10  forFollowUp            13810 non-null  object 
 11  operator               27303 non-null  object 
 12  origin                 26823 non-null  object 
 13  hazards                20241 non-null  object 
 14  year                   27397 non-null  int64  
 15  mo

##### Text vectorizing using TF-IDF

In [19]:
def clean_pipeline(text):
    text = text.lower().strip()
    
    # Keep numbers, hyphens, letters. Remove only noise punctuation.
    text = re.sub(r"[^a-z0-9\-\s/\.]", " ", text)
    
    # Normalize spaces
    text = re.sub(r"\s+", " ", text)
    
    return text

In [20]:
rasff['subject_clean'] = rasff['subject'].fillna('').apply(clean_pipeline)

In [23]:
rasff['subject_clean']

0        suspected salmonella i beef and hamburgers fro...
1        dna prze uwaczy w materiale paszowym dla wi he...
2        food supplements from usa containing prohibite...
3        presence of listeria monocytogenes st 451 in d...
4        food supplement with novel food ingredient fro...
                               ...                        
27392    migration of formaldehyde form bamboo mug from...
27393    salmonella spp. in 1 out of 5 samples in hot s...
27394             fosthiazate in fresh peppers from turkey
27395                 aflatoxins in dried figs from turkey
27396                 aflatoxins in dried figs from turkey
Name: subject_clean, Length: 27397, dtype: object

In [24]:
rasff['subject']

0        Suspected Salmonella i beef and hamburgers fro...
1        DNA przeżuwaczy w materiale paszowym dla świń ...
2        Food supplements from USA containing prohibite...
3        Presence of Listeria monocytogenes ST 451 in d...
4        Food supplement with novel food ingredient fro...
                               ...                        
27392    migration of formaldehyde form bamboo mug from...
27393    Salmonella spp. (in 1 out of 5 samples) in Hot...
27394             fosthiazate in fresh peppers from Turkey
27395                 aflatoxins in dried figs from Turkey
27396                Aflatoxins in dried figs from Turkey 
Name: subject, Length: 27397, dtype: object

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Initialize and Fit the TF-IDF Vectorizer ---

# TfidfVectorizer handles tokenization, counting, and calculating the
# TF-IDF score for each word (token) across all documents.
vectorizer = TfidfVectorizer(
    stop_words='english',  # Remove common words like 'the', 'a', 'concerning'
    max_features=2000,       # Increased the limit to 20 features (use 500-2000 for 27k rows)
    ngram_range=(2, 2),    # Include single words (unigrams) and two-word phrases (bigrams)
    min_df=10,              # A term must appear in at least 1 document (use 5-10 for 27k rows)
    max_df=0.95             # A term must appear in less than 100% of documents (use 0.90-0.95 for 27k rows)
)

In [26]:
# Fit the vectorizer to the text data and transform it into a sparse matrix
tfidf_matrix = vectorizer.fit_transform(rasff['subject'])

In [27]:

# Convert Results to a Readable DataFrame ---

# Get the feature names (the actual words/ngrams)
feature_names = vectorizer.get_feature_names_out()

# Convert the sparse matrix to a dense array and then to a DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=feature_names,
    index=rasff['reference']
)


In [28]:
# Display Results ---

print("--- TF-IDF Encoded Vectors (Top 20 Features) ---")
print("Each row (case) is now represented by 20 numerical scores.")
print(tfidf_df)
print("\n" + "="*50 + "\n")

# Show the highest scoring words and their importance (IDF)
print("--- Top TF-IDF Feature Names and their Inverse Document Frequency (IDF) ---")
idf_scores = pd.DataFrame({
    'term': feature_names,
    'idf_score': vectorizer.idf_
}).sort_values(by='idf_score', ascending=False)

print(idf_scores)

--- TF-IDF Encoded Vectors (Top 20 Features) ---
Each row (case) is now represented by 20 numerical scores.
           11 mg  110 sunset  12 mg  14 mg  171 titanium  2020 3678  \
reference                                                             
2025.8491    0.0         0.0    0.0    0.0           0.0        0.0   
2025.8490    0.0         0.0    0.0    0.0           0.0        0.0   
2025.8488    0.0         0.0    0.0    0.0           0.0        0.0   
2025.8487    0.0         0.0    0.0    0.0           0.0        0.0   
2025.8486    0.0         0.0    0.0    0.0           0.0        0.0   
...          ...         ...    ...    ...           ...        ...   
2020.0010    0.0         0.0    0.0    0.0           0.0        0.0   
2020.0008    0.0         0.0    0.0    0.0           0.0        0.0   
2020.0007    0.0         0.0    0.0    0.0           0.0        0.0   
2020.0004    0.0         0.0    0.0    0.0           0.0        0.0   
2020.0001    0.0         0.0    0.0    0

##### sentence embedding

In [29]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [30]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\SAMSUNG\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SAMSUNG\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
embeddings = model.encode(rasff["subject_clean"].tolist())

In [33]:
rasff.index

RangeIndex(start=0, stop=27397, step=1)

In [32]:
import numpy as np
emb_df = pd.DataFrame(embeddings, index=rasff.index)

In [34]:
rasff_emb=pd.concat([emb_df, rasff], axis=1)

In [37]:
rasff_emb.head()

,0,1,2,3,4,5,6,7,8,9,...,forFollowUp,operator,origin,hazards,year,month,Hazard_Type,Regulatory_Issue_Flag,subject_clena,subject_clean
0,0.023149,0.020542,-0.030111,-0.000022,0.058060,-0.045696,0.051135,0.065663,0.041160,-0.057489,...,"Denmark,Netherlands","Denmark,Netherlands,Sweden",Netherlands,NaN,2025,10,Microbiological,0,suspected salmonella beef and hamburgers fro d...,suspected salmonella i beef and hamburgers fro...
1,-0.016162,0.049640,-0.082455,-0.042600,-0.051902,0.000681,0.046981,0.024088,-0.032040,-0.004964,...,Belgium,"Belgium,Poland",Belgium,NaN,2025,10,Others,0,suspected salmonella beef and hamburgers fro d...,dna prze uwaczy w materiale paszowym dla wi he...
2,-0.041873,-0.004420,-0.063214,0.035958,0.022168,0.032789,0.003469,0.042927,-0.066122,-0.077469,...,United States,United States,United States,NaN,2025,10,Chemical/Contaminants,1,suspected salmonella beef and hamburgers fro d...,food supplements from usa containing prohibite...
3,-0.020285,-0.008071,0.069631,-0.009426,-0.082839,0.018862,-0.074184,0.029647,0.030166,-0.077143,...,"Austria,Germany,Netherlands,Slovenia","Austria,Italy",Italy,Listeria monocytogenes,2025,10,Microbiological,0,suspected salmonella beef and hamburgers fro d...,presence of listeria monocytogenes st 451 in d...
4,0.004986,-0.063098,0.000090,0.039775,-0.003351,0.065152,0.002741,-0.014909,-0.025817,-0.077155,...,United Kingdom,"Ireland,United Kingdom",United Kingdom,not in catalogue,2025,10,Novel/Unauthorised Ingredient,0,suspected salmonella beef and hamburgers fro d...,food supplement with novel food ingredient fro...


##### one-hot encoding on hazard_type

In [38]:
#Perform One-Hot Encoding on 'hazard_type' ---

# Use pandas' get_dummies function for OHE.
# 'prefix' sets the column names for the new dummy variables (e.g., 'hazard_type_Mycotoxin').
rasff_encoded = pd.get_dummies(rasff_emb, columns=['Hazard_Type'], prefix='hazard')



In [42]:
# --- 3. Display Results ---

print("--- Encoded DataFrame (After One-Hot Encoding 'hazard_type') ---")
print(rasff_encoded)
print("\n" + "="*50 + "\n")


--- Encoded DataFrame (After One-Hot Encoding 'hazard_type') ---
              0         1         2         3         4         5         6  \
0      0.023149  0.020542 -0.030111 -0.000022  0.058060 -0.045696  0.051135   
1     -0.016162  0.049640 -0.082455 -0.042600 -0.051902  0.000681  0.046981   
2     -0.041873 -0.004420 -0.063214  0.035958  0.022168  0.032789  0.003469   
3     -0.020285 -0.008071  0.069631 -0.009426 -0.082839  0.018862 -0.074184   
4      0.004986 -0.063098  0.000090  0.039775 -0.003351  0.065152  0.002741   
...         ...       ...       ...       ...       ...       ...       ...   
27392 -0.049578  0.022277  0.019573  0.040566  0.145016 -0.017994 -0.019010   
27393  0.047157 -0.010047 -0.024407  0.023813  0.030101 -0.009171  0.004004   
27394  0.008577  0.029172 -0.098910  0.072603  0.011805 -0.010723 -0.032932   
27395  0.087587  0.010154  0.006940  0.082257  0.064434  0.036325 -0.032588   
27396  0.087587  0.010154  0.006940  0.082257  0.064434  0.036325 

In [46]:
rasff_emb.head()

,0,1,2,3,4,5,6,7,8,9,...,forFollowUp,operator,origin,hazards,year,month,Hazard_Type,Regulatory_Issue_Flag,subject_clena,subject_clean
0,0.023149,0.020542,-0.030111,-0.000022,0.058060,-0.045696,0.051135,0.065663,0.041160,-0.057489,...,"Denmark,Netherlands","Denmark,Netherlands,Sweden",Netherlands,NaN,2025,10,Microbiological,0,suspected salmonella beef and hamburgers fro d...,suspected salmonella i beef and hamburgers fro...
1,-0.016162,0.049640,-0.082455,-0.042600,-0.051902,0.000681,0.046981,0.024088,-0.032040,-0.004964,...,Belgium,"Belgium,Poland",Belgium,NaN,2025,10,Others,0,suspected salmonella beef and hamburgers fro d...,dna prze uwaczy w materiale paszowym dla wi he...
2,-0.041873,-0.004420,-0.063214,0.035958,0.022168,0.032789,0.003469,0.042927,-0.066122,-0.077469,...,United States,United States,United States,NaN,2025,10,Chemical/Contaminants,1,suspected salmonella beef and hamburgers fro d...,food supplements from usa containing prohibite...
3,-0.020285,-0.008071,0.069631,-0.009426,-0.082839,0.018862,-0.074184,0.029647,0.030166,-0.077143,...,"Austria,Germany,Netherlands,Slovenia","Austria,Italy",Italy,Listeria monocytogenes,2025,10,Microbiological,0,suspected salmonella beef and hamburgers fro d...,presence of listeria monocytogenes st 451 in d...
4,0.004986,-0.063098,0.000090,0.039775,-0.003351,0.065152,0.002741,-0.014909,-0.025817,-0.077155,...,United Kingdom,"Ireland,United Kingdom",United Kingdom,not in catalogue,2025,10,Novel/Unauthorised Ingredient,0,suspected salmonella beef and hamburgers fro d...,food supplement with novel food ingredient fro...


In [48]:
rasff_encoded['Hazard_Type'] = rasff_emb['Hazard_Type']

In [49]:
rasff_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,hazard_Allergen,hazard_Chemical/Contaminants,hazard_Fraud/Adulteration,hazard_Microbiological,hazard_Migration from Packaging,hazard_Mycotoxins,hazard_Novel/Unauthorised Ingredient,hazard_Others,hazard_Physical Contaminants,Hazard_Type
0,0.023149,0.020542,-0.030111,-0.000022,0.058060,-0.045696,0.051135,0.065663,0.041160,-0.057489,...,False,False,False,True,False,False,False,False,False,Microbiological
1,-0.016162,0.049640,-0.082455,-0.042600,-0.051902,0.000681,0.046981,0.024088,-0.032040,-0.004964,...,False,False,False,False,False,False,False,True,False,Others
2,-0.041873,-0.004420,-0.063214,0.035958,0.022168,0.032789,0.003469,0.042927,-0.066122,-0.077469,...,False,True,False,False,False,False,False,False,False,Chemical/Contaminants
3,-0.020285,-0.008071,0.069631,-0.009426,-0.082839,0.018862,-0.074184,0.029647,0.030166,-0.077143,...,False,False,False,True,False,False,False,False,False,Microbiological
4,0.004986,-0.063098,0.000090,0.039775,-0.003351,0.065152,0.002741,-0.014909,-0.025817,-0.077155,...,False,False,False,False,False,False,True,False,False,Novel/Unauthorised Ingredient


In [50]:
rasff_encoded.to_csv("rasff_encoded.csv", index=False)

In [2]:
import pandas  as pd

In [3]:
rasff=pd.read_csv(r"F:\Final_project\rasff_encoded.csv")

In [10]:
rasff['risk_decision'].value_counts()

risk_decision
4    14756
1     4134
5     3029
2     2689
3     2372
0      417
Name: count, dtype: int64

##### target encoding on origin, category, notifying country

In [4]:
%pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from category_encoders import CatBoostEncoder

# --------------------------------------------------------
# 1. Define Columns
# --------------------------------------------------------
target_col = "risk_decision"
categorical_cols = ["origin", "category", "notifying_country"]

# Ensure target is numeric (CatBoostEncoder requires numeric target)
rasff[target_col] = rasff[target_col].astype("category").cat.codes

# --------------------------------------------------------
# 2. Initialize CatBoostEncoder
# --------------------------------------------------------
cbe = CatBoostEncoder(cols=categorical_cols)

# --------------------------------------------------------
# 3. Fit encoder on full dataset and transform
# --------------------------------------------------------
df_encoded = cbe.fit_transform(rasff[categorical_cols], rasff[target_col])

# --------------------------------------------------------
# 4. Rename encoded columns (optional but recommended)
# --------------------------------------------------------
df_encoded.columns = [f"{col}_te" for col in categorical_cols]

# --------------------------------------------------------
# 5. Merge encoded values back into your dataset
# --------------------------------------------------------
df_te = pd.concat([rasff, df_encoded], axis=1)

# --------------------------------------------------------
# 6. Display result
# --------------------------------------------------------
print(df_te.head())
print(df_te.columns)

          0         1         2         3         4         5         6  \
0  0.023149  0.020542 -0.030111 -0.000022  0.058060 -0.045696  0.051135   
1 -0.016162  0.049640 -0.082455 -0.042600 -0.051902  0.000681  0.046981   
2 -0.041873 -0.004420 -0.063214  0.035958  0.022168  0.032789  0.003469   
3 -0.020285 -0.008071  0.069631 -0.009426 -0.082839  0.018862 -0.074184   
4  0.004986 -0.063098  0.000090  0.039775 -0.003351  0.065152  0.002741   

          7         8         9  ...  hazard_Microbiological  \
0  0.065663  0.041160 -0.057489  ...                    True   
1  0.024088 -0.032040 -0.004964  ...                   False   
2  0.042927 -0.066122 -0.077469  ...                   False   
3  0.029647  0.030166 -0.077143  ...                    True   
4 -0.014909 -0.025817 -0.077155  ...                   False   

   hazard_Migration from Packaging  hazard_Mycotoxins  \
0                            False              False   
1                            False              Fa

In [6]:
df_te.head()

,0,1,2,3,4,5,6,7,8,9,...,hazard_Microbiological,hazard_Migration from Packaging,hazard_Mycotoxins,hazard_Novel/Unauthorised Ingredient,hazard_Others,hazard_Physical Contaminants,Hazard_Type,origin_te,category_te,notifying_country_te
0,0.023149,0.020542,-0.030111,-0.000022,0.058060,-0.045696,0.051135,0.065663,0.041160,-0.057489,...,True,False,False,False,False,False,Microbiological,3.314122,3.314122,3.314122
1,-0.016162,0.049640,-0.082455,-0.042600,-0.051902,0.000681,0.046981,0.024088,-0.032040,-0.004964,...,False,False,False,False,True,False,Others,3.314122,3.314122,3.314122
2,-0.041873,-0.004420,-0.063214,0.035958,0.022168,0.032789,0.003469,0.042927,-0.066122,-0.077469,...,False,False,False,False,False,False,Chemical/Contaminants,3.314122,3.314122,3.314122
3,-0.020285,-0.008071,0.069631,-0.009426,-0.082839,0.018862,-0.074184,0.029647,0.030166,-0.077143,...,True,False,False,False,False,False,Microbiological,3.314122,2.657061,3.314122
4,0.004986,-0.063098,0.000090,0.039775,-0.003351,0.065152,0.002741,-0.014909,-0.025817,-0.077155,...,False,False,False,True,False,False,Novel/Unauthorised Ingredient,3.314122,2.657061,3.314122


##### how to handle 'classificaiton' column?

In [7]:
df_te.groupby(["classification", "risk_decision"]).size()

classification                          risk_decision
alert notification                      0                   4
                                        1                   7
                                        2                  45
                                        3                 581
                                        4                7028
                                        5                 330
border rejection notification           0                 334
                                        1                1414
                                        2                 762
                                        3                 560
                                        4                4284
                                        5                1066
information notification for attention  0                  31
                                        1                 920
                                        2                 646
                